https://www.metacritic.com/game 에서 크롤링 예정

In [1]:
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

헤더부분 유저에이전트,레퍼러 예제

In [2]:
hur={'User-Agent':'Mozilla','referer':'https://www.metacritic.com/game/playstation-5/elden-ring'}
url='https://www.metacritic.com/game/playstation-5/elden-ring'
req=requests.get(url,headers=hur)
bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8') #beautifulsoup객체선언
q1=bsoup.find("div",{"class":"details main_details"})
temp=bsoup.find("span",{"class":"blurb blurb_expanded"}).text
print(temp)

A New World Created By Hidetaka Miyazaki And George R. R. Martin. ELDEN RING, developed by FromSoftware, Inc. and BANDAI NAMCO Entertainment Inc., is a fantasy action-RPG adventure set within a world created by Hidetaka Miyazaki creator of the influential DARK SOULS video game series; and George R.R. Martin author of The New York Times best-selling fantasy series, A Song of Ice and Fire. Danger and discovery lurk around every corner in FromSoftware's largest game to-date.  Hidetaka Miyazaki, President and Game Director of FromSoftware Inc. Known for directing critically-acclaimed games in beloved franchises including Armored Core, Dark Souls, and Sekiro: Shadows Die Twice.  George R.R. Martin is the #1 New York Times bestselling author of many novels, including the acclaimed series A Song of Ice and Fire - A Game of Thrones, A Clash of Kings, A Storm of Swords, A Feast For Crows, and A Dance with Dragons. As a writer-producer, he has worked on The Twilight Zone, Beauty and the Beast, a

이전까지 크롤링했던 진도부터 하기

정해진 플랫폼에 따라 크롤링하는 함수

In [7]:
def crawl_page(url):
    ret={}
    chk=True
    print(url)
    cnt=0
    while(chk):
        req=requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36','referer':url}) 
        time.sleep(2.1)#크롤링 속도 제한
        if(req==None): 
            print("reqNone")
            continue
        bsoup=BeautifulSoup(req.text,'lxml')
        if(bsoup==None):
            print("bsoupNone")
            continue
        bmetascore=bsoup.find("span",{"itemprop":"ratingValue"})
        if(bmetascore==None):
            time.sleep(4)
            print("bemtaNone")
            cnt+=1;
            if(cnt>=5):
                cnt=0;
                return False;
            continue
        bsummary=bsoup.find("span",class_="blurb blurb_expanded")
        if(bsummary==None):
            print("bsumm")
            bsummary="None"
        else:
            if(bsummary.string==None):
                bsummary=bsoup.find("span",class_="blurb blurb_collapsed")
                if(bsummary.string!=None):
                    bsummary=bsummary.string
                if(bsummary.string!=None):
                    bsummary=bsummary.string
            else:
                bsummary=bsummary.string
        btitle=bsoup.find("div",{"class":"product_title"}).find("h1")
        if(btitle==None): 
            print("btitleNone")
            continue
        buserscore=bsoup.find("div",class_=re.compile("metascore_w user large game"))
        if(buserscore==None): 
            print("buserNone")
            buserscore="None"
        else:
            buserscore=buserscore.string
        bpub=bsoup.find("li",class_="summary_detail publisher").find("a")
        if(bpub==None):
            print("bpubNone")
            continue
        bdev=(bsoup.find("li",class_="summary_detail developer"))
        if(bdev==None):
            bdev="None"
            print("bdevNone")
        else:
            bdev=bdev.find("a")
            bdev=bdev.string
        bgenres=(bsoup.find("li",class_="summary_detail product_genre"))
        if(bgenres==None):
            print("bgenreNone")
            continue
        bgenres=bgenres.find_all("span",class_="data")
        sgenres=""
        for x in range(len(bgenres)):
            sgenres=sgenres+str(bgenres[x].string)+"|"
        bgrating=bsoup.find("li",class_="summary_detail product_rating")
        if(bgrating==None):
            print("bgratingNone")
            bgrating="None"
        else:
            bgrating=bgrating.find("span",class_="data").string
        brelease=bsoup.find("li",class_="summary_detail release_data")
        brelease=brelease.find("span",class_="data")
        print(str(btitle.string)," ",str(brelease.string)," ",str(bmetascore.string)," ",str(buserscore)," ",str(bpub.string)[57:-53]," ",str(bdev)," ",sgenres," ",bgrating)
        ret={"title":str(btitle.string),"metascore":str(bmetascore.string),"userscore":str(buserscore),"publisher":str(bpub.string)[57:-53],"developer":str(bdev),"genres":sgenres,"rating":str(bgrating),"summary":str(bsummary),"release_date":str(brelease.string)}
        chk=False
    return ret
    
def get_page(url):
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url})  
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    bsoup2=bsoup.find_all("td",{"class":"clamp-summary-wrap"})
    lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    for x in bsoup2:
        y=x.find("a")["href"]
        y=y[:-15]
        h=crawl_page("https://www.metacritic.com/"+y)
        if(h==False):
            continue
        df_dic=pd.DataFrame([h])
        lpd = pd.concat([lpd, df_dic], ignore_index=True)
    return lpd
        
def crawl_direct(platform,df):
    url="https://www.metacritic.com/browse/games/release-date/available/"+platform+"/metascore?page="
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url+'0'})
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    lpage_soup=bsoup.find("li",{"class":"last_page"})
    lpage2_soup=lpage_soup.find("a",{"class":"page_num"})
    lpagenum=lpage2_soup.text
    try:
        lpd=pd.read_csv(platform+'.csv')
    except:
        lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    f=open(platform+"_proc.txt","r")
    proc=int(f.readline())
    f.close()
    for x in range(proc,int(lpagenum)):  
        print(url+str(x))
        page=get_page(url+str(x))
        lpd=pd.concat([page,lpd],ignore_index=True)
        lpd.to_csv(platform+'.csv',sep=',',na_rep='NaN',float_format = '%.2f',index = False)
        f=open(platform+"_proc.txt","w")
        f.write(str(x+1))
        f.close()
        print(len(lpd))
crawl_direct('ps4',1)
crawl_direct('ps5',1)

https://www.metacritic.com/browse/games/release-date/available/ps5/metascore?page=1
https://www.metacritic.com//game/playstation-5/back-4-blood
Back 4 Blood   Oct  7, 2021   77   5.4   Warner Bros. Games   Turtle Rock Studios   Action|Shooter|First-Person|Arcade|   M
https://www.metacritic.com//game/playstation-5/nobody-saves-the-world
Nobody Saves the World   Apr 14, 2022   77   6.0   DrinkBox Studios   DrinkBox Studios   Role-Playing|Action RPG|   T
https://www.metacritic.com//game/playstation-5/cyber-shadow
Cyber Shadow   Jan 26, 2021   77   6.6   Yacht Club Games   Yacht Club Games   Action|Platformer|2D|   E10+
https://www.metacritic.com//game/playstation-5/observer-system-redux
Observer: System Redux   Nov 12, 2020   77   7.1   Aspyr   Bloober Team   Adventure|3D|First-Person|   M
https://www.metacritic.com//game/playstation-5/destiny-2-beyond-light
bgratingNone
Destiny 2: Beyond Light   Nov 10, 2020   77   5.8   Activision   Bungie Software   Action|Shooter|First-Person|Arcade| 

Monster Energy Supercross - The Official Videogame 5   Mar 17, 2022   74   3.9   Deep Silver   Milestone S.r.l   Racing|Simulation|Arcade|Automobile|   E
https://www.metacritic.com//game/playstation-5/mortal-shell-enhanced-edition
bgratingNone
Mortal Shell: Enhanced Edition   Mar  4, 2021   74   6.5   Playstack   Cold Symmetry   Role-Playing|Action RPG|   None
https://www.metacritic.com//game/playstation-5/vampire-the-masquerade---bloodhunt
Vampire: The Masquerade - Bloodhunt   Apr 27, 2022   74   6.2   Sharkmob   Sharkmob   Role-Playing|Western-Style|Action|Shooter|Third-Person|Tactical|   M
https://www.metacritic.com//game/playstation-5/fifa-21-nxt-lvl-edition
FIFA 21 NXT LVL EDITION   Dec  4, 2020   74   2.3   Electronic Arts   Electronic Arts   Sports|Team|Soccer|Sim|   E
https://www.metacritic.com//game/playstation-5/tom-clancys-rainbow-six-extraction
Tom Clancy's Rainbow Six Extraction   Jan 20, 2022   73   5.2   Ubisoft   Ubisoft Montreal   Action|Shooter|First-Person|Tactical| 

bsumm
Lawn Mowing Simulator   Mar 31, 2022   70   tbd   Curve Digital   Skyhook Games   Simulation|Virtual|Career|   E
https://www.metacritic.com//game/playstation-5/assassins-creed-valhalla-wrath-of-the-druids
bgratingNone
Assassin's Creed Valhalla: Wrath of the Druids   May 13, 2021   70   4.6   Ubisoft   Ubisoft Bordeaux   Action Adventure|Open-World|   None
https://www.metacritic.com//game/playstation-5/trek-to-yomi
Trek to Yomi   May  5, 2022   70   7.0   Devolver Digital   Flying Wild Hog   Action Adventure|General|   M
https://www.metacritic.com//game/playstation-5/maquette
bgratingNone
Maquette   Mar  2, 2021   70   6.2   Annapurna Interactive   Graceful Decay   Puzzle|General|   None
https://www.metacritic.com//game/playstation-5/demon-slayer-kimetsu-no-yaiba---the-hinokami-chronicles
bgratingNone
Demon Slayer: Kimetsu no Yaiba - The Hinokami Chronicles   Oct 13, 2021   70   7.7   Sega   CyberConnect2   Action|Fighting|3D|   None
https://www.metacritic.com//game/playstation-5/

Battlefield 2042   Nov 12, 2021   63   2.6   Electronic Arts   EA DICE   Action|Shooter|First-Person|Tactical|   M
https://www.metacritic.com//game/playstation-5/astria-ascending
Astria Ascending   Sep 30, 2021   63   6.8   Dear Villagers   Artisan Studios   Role-Playing|Japanese-Style|   T
https://www.metacritic.com//game/playstation-5/cricket-22
Cricket 22   Dec  2, 2021   63   5.8   Big Ant Studios   Big Ant Studios   Sports|Team|Cricket|   E
https://www.metacritic.com//game/playstation-5/backbone
Backbone   Oct 28, 2021   63   tbd   Raw Fury   EggNut   Adventure|General|   M
https://www.metacritic.com//game/playstation-5/destruction-allstars
Destruction AllStars   Feb  2, 2021   62   5.0   Sony Interactive Entertainment   Lucid   Simulation|Vehicle|Combat|   T
https://www.metacritic.com//game/playstation-5/terminator-resistance-enhanced
Terminator: Resistance Enhanced   Apr 30, 2021   62   7.0   Reef Entertainment   Teyon   Action|Shooter|First-Person|Arcade|   M
https://www.metacr